### **Gather The Data**
> **[``31-07-2025``]()** $\rightarrow$ **[``01-08-2025``]()**

In [ ]:
from transformers import AutoProcessor, AutoModelForCausalLM
from flask import Flask, request, jsonify, send_file
from typing import Dict, List, Tuple
from datetime import datetime
from pathlib import Path
from queue import Queue
from PIL import Image

import matplotlib.pyplot as plt
import supervision as sv
import numpy as np
import threading
import warnings
import logging
import sqlite3
import base64
import torch
import json
import time
import sys
import io
import os

# Configurações
BASE_DATA_DIR = "GhData"
IMAGE_QUALITY = 95
USE_GPU = False 
device = 'cuda' if (USE_GPU and torch.cuda.is_available()) else 'cpu'
processing_queue = Queue()

# Logging configuration
os.makedirs('logs', exist_ok=True)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(os.path.join('logs', 'overdataserver.log')),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)
warnings.filterwarnings('ignore')
app = Flask(__name__)

# Modelo: Microsoft Florense-2 (Pré-treinado + fine-tuned)
model_id = 'microsoft/Florence-2-base' # 'florence-2-large-ft' | 'florence-2-base-ft' (revision='refs/pr/6' only ft)
florence_processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
florence_model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, torch_dtype=torch.float32, _attn_implementation="eager").to(device)

# Modelo: GroundingDINO
HOME = os.getcwd()
CONFIG_PATH = os.path.join(HOME, 'GroundingDINO', 'groundingdino', 'config', 'GroundingDINO_SwinB_cfg.py') # BASE: GroundingDINO_SwinT_OGC.py
os.chdir(os.path.join(HOME, 'GroundingDINO'))
WEIGHTS_NAME = 'groundingdino_swinb_cogcoor.pth' # BASE: groundingdino_swint_ogc.pth
WEIGHTS_PATH = os.path.join(HOME, 'GroundingDINO', 'weights', WEIGHTS_NAME)
from groundingdino.util.inference import load_model, load_image, predict, annotate
gd_model = load_model(CONFIG_PATH, WEIGHTS_PATH)

os.chdir(HOME)

In [2]:
def seeknow_analysis(image_base64: str, list_prompt: list, title_prompt: str = "", box_threshold=0.35, text_threshold=0.25) -> dict:
    """
    Perform object detection on a base64 encoded image using GroundingDINO model.
    
    Args:
        image_base64 (str): Base64 encoded image string
        list_prompt (list): List of objects to detect in the image
        title_prompt (str, optional): Title to return if objects are detected. Defaults to "".
        box_threshold (float, optional): Confidence threshold for bounding boxes. Defaults to 0.35.
        text_threshold (float, optional): Confidence threshold for text matching. Defaults to 0.25.
    
    Returns:
        dict: Dictionary containing:
            - image (str): Base64 encoded annotated image
            - title_prompt_found (str|bool): Provided title if objects detected, else False
            - detections (dict): Dictionary with:
                - boxes (list): List of detected bounding boxes in pixel coordinates
                - scores (list): Confidence scores for each detection
                - labels (list): Detected object labels
            - execution_time (float): Total processing time in seconds
    """
    global gd_model
    
    start_time = time.time()
    
    # Process image
    image = Image.open(io.BytesIO(base64.b64decode(image_base64)))
    image_np = np.array(image)
    h, w = image_np.shape[:2]
    
    # Run prediction
    boxes, logits, phrases = predict(
        model=gd_model,
        image=torch.from_numpy(image_np).permute(2, 0, 1).float().to(device),
        caption=', '.join(list_prompt),
        box_threshold=box_threshold,
        text_threshold=text_threshold,
        device=device
    )
    
    # Convert boxes to pixel coords
    pixel_boxes = []
    has_boxes = False
    if boxes is not None:
        boxes_list = boxes.tolist() if hasattr(boxes, 'tolist') else boxes
        if len(boxes_list) > 0:
            has_boxes = True
            for x, y, bw, bh in boxes_list:
                x1, y1 = int((x-bw/2)*w), int((y-bh/2)*h)
                x2, y2 = int((x+bw/2)*w), int((y+bh/2)*h)
                pixel_boxes.append([x1, y1, x2, y2])
    
    # Annotate and encode image
    buffered = io.BytesIO()
    Image.fromarray(annotate(image_np, boxes, logits, phrases)[..., ::-1]).save(buffered, format="JPEG", quality=90)
    
    execution_time = time.time() - start_time
    
    return {
        "image": base64.b64encode(buffered.getvalue()).decode(),
        "title_prompt_found": title_prompt if has_boxes else False,
        "detections": {
            "boxes": pixel_boxes,
            "scores": [round(score, 4) for score in logits.tolist()] if logits is not None else [],
            "labels": phrases if phrases is not None else []
        },
        "execution_time": execution_time
    }

In [3]:
def extractinfo_analysis(image_base64: str, list_prompt: list = []) -> Dict:
    '''Gera captions usando Florence-2 a partir de imagem em base64'''
    global florence_model, florence_processor, device

    start_time = time.time()

    # Convert base64 to PIL Image
    image = Image.open(io.BytesIO(base64.b64decode(image_base64)))
    img_np = np.array(image)

    prompts = [
        '<CAPTION_TO_PHRASE_GROUNDING>',
        '<CAPTION>',
        '<DETAILED_CAPTION>', 
        '<MORE_DETAILED_CAPTION>',
        '<DENSE_REGION_CAPTION>',
        '<OD>',
    ]

    results = {
        'execution_times': {},
        'total_time': 0,
        'annotated_images': {}
    }
    
    for prompt in prompts:
        prompt_start = time.time()
        
        prompt_text = prompt if prompt != '<CAPTION_TO_PHRASE_GROUNDING>' else prompt+' '.join([f'{x}.' for x in list_prompt])
        inputs = florence_processor(text=prompt_text, images=image, return_tensors='pt').to(device)
        
        generated_ids = florence_model.generate(
            input_ids=inputs['input_ids'],
            pixel_values=inputs['pixel_values'],
            max_new_tokens=1024,
            do_sample=False,
            num_beams=3,
            early_stopping=False,
            use_cache=False,
        )
        generated_text = florence_processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
        parsed_answer = florence_processor.post_process_generation(
            generated_text,
            task=prompt,
            image_size=(image.width, image.height)
        )
        
        prompt_time = time.time() - prompt_start
        results[prompt] = parsed_answer.get(prompt, '')
        results['execution_times'][prompt] = round(prompt_time, 2)
    
    results['total_time'] = round(time.time() - start_time, 2)

    def add_annotations(img_np, bboxes, labels, color):
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.imshow(img_np)
        for bbox, label in zip(bboxes, labels):
            x, y, w, h = bbox
            rect = plt.Rectangle((x, y), w-x, h-y, linewidth=1, 
                                edgecolor=color, facecolor='none')
            ax.add_patch(rect)
            ax.text(x, y, label, color='white', backgroundcolor=color)
        plt.axis('off')
        buf = io.BytesIO()
        plt.savefig(buf, format='jpg', bbox_inches='tight', pad_inches=0)
        plt.close()
        buf.seek(0)
        return base64.b64encode(buf.getvalue()).decode('utf-8')

    annotated_images = {}

    if '<DENSE_REGION_CAPTION>' in results and 'bboxes' in results['<DENSE_REGION_CAPTION>']:
        annotated_images['dense_region_caption'] = add_annotations(
            img_np, 
            results['<DENSE_REGION_CAPTION>']['bboxes'],
            results['<DENSE_REGION_CAPTION>']['labels'],
            '#a83232'
        )

    if '<OD>' in results and 'bboxes' in results['<OD>']:
        annotated_images['od'] = add_annotations(
            img_np,
            results['<OD>']['bboxes'],
            results['<OD>']['labels'], 
            '#3251a8'
        )

    if '<CAPTION_TO_PHRASE_GROUNDING>' in results and 'bboxes' in results['<CAPTION_TO_PHRASE_GROUNDING>']:
        annotated_images['phrase_grounding'] = add_annotations(
            img_np,
            results['<CAPTION_TO_PHRASE_GROUNDING>']['bboxes'],
            results['<CAPTION_TO_PHRASE_GROUNDING>']['labels'],
            '#5f32a8'
        )

    results['annotated_images'] = annotated_images
    return results

In [4]:
with open('/home/gabrielskaftell/Downloads/Pasted image (2).png', "rb") as image_file:
    image_base64 = base64.b64encode(image_file.read()).decode('utf-8')

# result_gd = seeknow_analysis(image_base64, list_prompt=['dog', 'hotdog', 'shoes'], title_prompt='example')
# result_fl = extractinfo_analysis(image_base64=image_base64, list_prompt=['dog', 'hotdog', 'shoes', 'trees'])

In [5]:
class DatabaseManager:
    """Gerenciador de banco de dados SQLite para cada empresa"""
    
    def __init__(self, company_id: str):
        self.company_id = company_id
        self.db_path = os.path.join(BASE_DATA_DIR, company_id, f"{company_id}_database.db")
        self._create_tables()
    
    def _create_tables(self):
        """Cria as tabelas necessárias no banco de dados"""
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            
            # Tabela de empresas
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS companies (
                    id TEXT PRIMARY KEY,
                    name TEXT NOT NULL,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            ''')
            
            # Tabela de coletores
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS collectors (
                    id TEXT PRIMARY KEY,
                    company_id TEXT NOT NULL,
                    model TEXT NOT NULL,
                    description TEXT,
                    latitude REAL,
                    longitude REAL,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    FOREIGN KEY (company_id) REFERENCES companies (id)
                )
            ''')
            
            # Tabela de capturas
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS captures (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    company_id TEXT NOT NULL,
                    collector_id TEXT NOT NULL,
                    timestamp TEXT NOT NULL,
                    raw_image_path TEXT NOT NULL,
                    processed_image_path TEXT NOT NULL,
                    raw_image_size_mb REAL,
                    processed_image_size_mb REAL,
                    camera_settings TEXT,
                    system_metadata TEXT,
                    system_status TEXT,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    FOREIGN KEY (company_id) REFERENCES companies (id),
                    FOREIGN KEY (collector_id) REFERENCES collectors (id)
                )
            ''')
            
            # Tabela de detecções originais (do coletor)
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS original_detections (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    capture_id INTEGER NOT NULL,
                    x INTEGER NOT NULL,
                    y INTEGER NOT NULL,
                    width INTEGER NOT NULL,
                    height INTEGER NOT NULL,
                    confidence REAL NOT NULL,
                    class_name TEXT NOT NULL,
                    class_id INTEGER NOT NULL,
                    FOREIGN KEY (capture_id) REFERENCES captures (id)
                )
            ''')
            
            # Tabela de análises Grounding DINO
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS grounding_dino_analysis (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    capture_id INTEGER NOT NULL,
                    prompt_title TEXT NOT NULL,
                    user_prompts TEXT NOT NULL,
                    image_path TEXT NOT NULL,
                    execution_time REAL NOT NULL,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    FOREIGN KEY (capture_id) REFERENCES captures (id)
                )
            ''')
            
            # Tabela de detecções Grounding DINO
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS grounding_dino_detections (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    analysis_id INTEGER NOT NULL,
                    x INTEGER NOT NULL,
                    y INTEGER NOT NULL,
                    width INTEGER NOT NULL,
                    height INTEGER NOT NULL,
                    score REAL NOT NULL,
                    label TEXT NOT NULL,
                    FOREIGN KEY (analysis_id) REFERENCES grounding_dino_analysis (id)
                )
            ''')
            
            # Tabela de análises Florence-2
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS florence2_analysis (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    capture_id INTEGER NOT NULL,
                    total_execution_time REAL NOT NULL,
                    caption TEXT,
                    detailed_caption TEXT,
                    more_detailed_caption TEXT,
                    dense_region_image_path TEXT,
                    od_image_path TEXT,
                    phrase_grounding_image_path TEXT,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    FOREIGN KEY (capture_id) REFERENCES captures (id)
                )
            ''')
            
            # Tabela de tempos de execução Florence-2
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS florence2_execution_times (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    analysis_id INTEGER NOT NULL,
                    task_name TEXT NOT NULL,
                    execution_time REAL NOT NULL,
                    FOREIGN KEY (analysis_id) REFERENCES florence2_analysis (id)
                )
            ''')
            
            # Tabela de detecções Florence-2 (Caption to Phrase Grounding)
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS florence2_phrase_detections (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    analysis_id INTEGER NOT NULL,
                    x1 REAL NOT NULL,
                    y1 REAL NOT NULL,
                    x2 REAL NOT NULL,
                    y2 REAL NOT NULL,
                    label TEXT NOT NULL,
                    FOREIGN KEY (analysis_id) REFERENCES florence2_analysis (id)
                )
            ''')
            
            # Tabela de regiões densas Florence-2
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS florence2_dense_regions (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    analysis_id INTEGER NOT NULL,
                    x1 REAL NOT NULL,
                    y1 REAL NOT NULL,
                    x2 REAL NOT NULL,
                    y2 REAL NOT NULL,
                    label TEXT NOT NULL,
                    FOREIGN KEY (analysis_id) REFERENCES florence2_analysis (id)
                )
            ''')
            
            # Tabela de detecções de objetos Florence-2
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS florence2_object_detections (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    analysis_id INTEGER NOT NULL,
                    x1 REAL NOT NULL,
                    y1 REAL NOT NULL,
                    x2 REAL NOT NULL,
                    y2 REAL NOT NULL,
                    label TEXT NOT NULL,
                    FOREIGN KEY (analysis_id) REFERENCES florence2_analysis (id)
                )
            ''')
            
            conn.commit()
    
    def insert_company(self, company_id: str, company_name: str):
        """Insere ou atualiza informações da empresa"""
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute('''
                INSERT OR REPLACE INTO companies (id, name)
                VALUES (?, ?)
            ''', (company_id, company_name))
            conn.commit()
    
    def insert_collector(self, collector_data: dict, company_id: str):
        """Insere ou atualiza informações do coletor"""
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute('''
                INSERT OR REPLACE INTO collectors 
                (id, company_id, model, description, latitude, longitude)
                VALUES (?, ?, ?, ?, ?, ?)
            ''', (
                collector_data['id'],
                company_id,
                collector_data['model'],
                collector_data.get('description', ''),
                collector_data.get('latitude'),
                collector_data.get('longitude')
            ))
            conn.commit()
    
    def insert_capture(self, capture_data: dict) -> int:
        """Insere dados da captura e retorna o ID"""
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute('''
                INSERT INTO captures 
                (company_id, collector_id, timestamp, raw_image_path, processed_image_path,
                 raw_image_size_mb, processed_image_size_mb, camera_settings, 
                 system_metadata, system_status)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                capture_data['company_id'],
                capture_data['collector_id'],
                capture_data['timestamp'],
                capture_data['raw_image_path'],
                capture_data['processed_image_path'],
                capture_data['raw_image_size_mb'],
                capture_data['processed_image_size_mb'],
                json.dumps(capture_data['camera_settings']),
                json.dumps(capture_data['system_metadata']),
                json.dumps(capture_data['system_status'])
            ))
            conn.commit()
            return cursor.lastrowid
    
    def insert_original_detections(self, capture_id: int, detections: List[dict]):
        """Insere detecções originais do coletor"""
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            for detection in detections:
                cursor.execute('''
                    INSERT INTO original_detections 
                    (capture_id, x, y, width, height, confidence, class_name, class_id)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                ''', (
                    capture_id,
                    detection['x'],
                    detection['y'],
                    detection['width'],
                    detection['height'],
                    detection['confidence'],
                    detection['class'],
                    detection['class_id']
                ))
            conn.commit()
    
    def insert_grounding_dino_analysis(self, capture_id: int, analysis_data: dict) -> int:
        """Insere análise do Grounding DINO e retorna o ID"""
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute('''
                INSERT INTO grounding_dino_analysis 
                (capture_id, prompt_title, user_prompts, image_path, execution_time)
                VALUES (?, ?, ?, ?, ?)
            ''', (
                capture_id,
                analysis_data['title_prompt_found'],
                json.dumps(analysis_data['user_prompts']),
                analysis_data['image_path'],
                analysis_data['execution_time']
            ))
            conn.commit()
            return cursor.lastrowid
    
    def insert_grounding_dino_detections(self, analysis_id: int, detections: dict):
        """Insere detecções do Grounding DINO"""
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            boxes = detections['boxes']
            scores = detections['scores']
            labels = detections['labels']
            
            for i, box in enumerate(boxes):
                cursor.execute('''
                    INSERT INTO grounding_dino_detections 
                    (analysis_id, x, y, width, height, score, label)
                    VALUES (?, ?, ?, ?, ?, ?, ?)
                ''', (
                    analysis_id,
                    box[0], box[1], box[2] - box[0], box[3] - box[1],
                    scores[i],
                    labels[i]
                ))
            conn.commit()
    
    def insert_florence2_analysis(self, capture_id: int, analysis_data: dict) -> int:
        """Insere análise do Florence-2 e retorna o ID"""
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute('''
                INSERT INTO florence2_analysis 
                (capture_id, total_execution_time, caption, detailed_caption, 
                 more_detailed_caption, dense_region_image_path, od_image_path, 
                 phrase_grounding_image_path)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                capture_id,
                analysis_data['total_time'],
                analysis_data.get('<CAPTION>', ''),
                analysis_data.get('<DETAILED_CAPTION>', ''),
                analysis_data.get('<MORE_DETAILED_CAPTION>', ''),
                analysis_data['image_paths']['dense_region_caption'],
                analysis_data['image_paths']['od'],
                analysis_data['image_paths']['phrase_grounding']
            ))
            conn.commit()
            return cursor.lastrowid
    
    def insert_florence2_execution_times(self, analysis_id: int, execution_times: dict):
        """Insere tempos de execução do Florence-2"""
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            for task_name, exec_time in execution_times.items():
                cursor.execute('''
                    INSERT INTO florence2_execution_times 
                    (analysis_id, task_name, execution_time)
                    VALUES (?, ?, ?)
                ''', (analysis_id, task_name, exec_time))
            conn.commit()
    
    def insert_florence2_detections(self, analysis_id: int, analysis_data: dict):
        """Insere todas as detecções do Florence-2"""
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            
            # Caption to Phrase Grounding
            if '<CAPTION_TO_PHRASE_GROUNDING>' in analysis_data:
                grounding_data = analysis_data['<CAPTION_TO_PHRASE_GROUNDING>']
                for i, bbox in enumerate(grounding_data['bboxes']):
                    cursor.execute('''
                        INSERT INTO florence2_phrase_detections 
                        (analysis_id, x1, y1, x2, y2, label)
                        VALUES (?, ?, ?, ?, ?, ?)
                    ''', (analysis_id, bbox[0], bbox[1], bbox[2], bbox[3], 
                          grounding_data['labels'][i]))
            
            # Dense Region Caption
            if '<DENSE_REGION_CAPTION>' in analysis_data:
                dense_data = analysis_data['<DENSE_REGION_CAPTION>']
                for i, bbox in enumerate(dense_data['bboxes']):
                    cursor.execute('''
                        INSERT INTO florence2_dense_regions 
                        (analysis_id, x1, y1, x2, y2, label)
                        VALUES (?, ?, ?, ?, ?, ?)
                    ''', (analysis_id, bbox[0], bbox[1], bbox[2], bbox[3], 
                          dense_data['labels'][i]))
            
            # Object Detection
            if '<OD>' in analysis_data:
                od_data = analysis_data['<OD>']
                for i, bbox in enumerate(od_data['bboxes']):
                    cursor.execute('''
                        INSERT INTO florence2_object_detections 
                        (analysis_id, x1, y1, x2, y2, label)
                        VALUES (?, ?, ?, ?, ?, ?)
                    ''', (analysis_id, bbox[0], bbox[1], bbox[2], bbox[3], 
                          od_data['labels'][i]))
            
            conn.commit()

In [6]:
class DirectoryManager:
    """Gerenciador de estrutura de diretórios"""
    
    @staticmethod
    def create_company_structure(company_id: str, collector_id: str):
        """Cria estrutura de diretórios para empresa e coletor"""
        base_path = Path(BASE_DATA_DIR) / company_id
        collector_path = base_path / collector_id
        
        # Criar diretórios
        directories = [
            base_path,
            collector_path,
            collector_path / "raw_capture",
            collector_path / "pyl_model",
            collector_path / "fld_model",
            collector_path / "flo_model", 
            collector_path / "flp_model",
            collector_path / "grd_model"
        ]
        
        for directory in directories:
            directory.mkdir(parents=True, exist_ok=True)
        
        return collector_path
    
    @staticmethod
    def generate_filename(prefix: str, company_id: str, collector_id: str, 
                         model: str, timestamp: str) -> str:
        """Gera nome do arquivo seguindo o padrão especificado"""
        return f"{prefix}-{company_id}-{collector_id}-{model}-{timestamp}.jpg"
    
    @staticmethod
    def save_base64_image(base64_data: str, file_path: str) -> bool:
        """Salva imagem base64 como arquivo JPG"""
        try:
            # Remove o prefixo data:image/... se existir
            if ',' in base64_data:
                base64_data = base64_data.split(',')[1]
            
            image_data = base64.b64decode(base64_data)
            
            with open(file_path, 'wb') as f:
                f.write(image_data)
            
            return True
        except Exception as e:
            logger.error(f"Erro ao salvar imagem {file_path}: {str(e)}")
            return False

In [7]:
class ModelAnalyzer:
    """Classe para análise com modelos (espaço reservado para suas funções)"""
    
    @staticmethod
    def seeknow_analysis(image_base64: str, list_prompt: list, title_prompt: str = "", box_threshold=0.35, text_threshold=0.25) -> dict:
        """
        Perform object detection on a base64 encoded image using GroundingDINO model.
        
        Args:
            image_base64 (str): Base64 encoded image string
            list_prompt (list): List of objects to detect in the image
            title_prompt (str, optional): Title to return if objects are detected. Defaults to "".
            box_threshold (float, optional): Confidence threshold for bounding boxes. Defaults to 0.35.
            text_threshold (float, optional): Confidence threshold for text matching. Defaults to 0.25.
        
        Returns:
            dict: Dictionary containing:
                - image (str): Base64 encoded annotated image
                - title_prompt_found (str|bool): Provided title if objects detected, else False
                - detections (dict): Dictionary with:
                    - boxes (list): List of detected bounding boxes in pixel coordinates
                    - scores (list): Confidence scores for each detection
                    - labels (list): Detected object labels
                - execution_time (float): Total processing time in seconds
        """
        global gd_model
        
        start_time = time.time()
        
        # Process image
        image = Image.open(io.BytesIO(base64.b64decode(image_base64)))
        image_np = np.array(image)
        h, w = image_np.shape[:2]
        
        # Run prediction
        boxes, logits, phrases = predict(
            model=gd_model,
            image=torch.from_numpy(image_np).permute(2, 0, 1).float().to(device),
            caption=', '.join(list_prompt),
            box_threshold=box_threshold,
            text_threshold=text_threshold,
            device=device
        )
        
        # Convert boxes to pixel coords
        pixel_boxes = []
        has_boxes = False
        if boxes is not None:
            boxes_list = boxes.tolist() if hasattr(boxes, 'tolist') else boxes
            if len(boxes_list) > 0:
                has_boxes = True
                for x, y, bw, bh in boxes_list:
                    x1, y1 = int((x-bw/2)*w), int((y-bh/2)*h)
                    x2, y2 = int((x+bw/2)*w), int((y+bh/2)*h)
                    pixel_boxes.append([x1, y1, x2, y2])
        
        # Annotate and encode image
        buffered = io.BytesIO()
        Image.fromarray(annotate(image_np, boxes, logits, phrases)[..., ::-1]).save(buffered, format="JPEG", quality=90)
        
        execution_time = time.time() - start_time
        
        return {
            "image": base64.b64encode(buffered.getvalue()).decode(),
            "title_prompt_found": title_prompt if has_boxes else False,
            "detections": {
                "boxes": pixel_boxes,
                "scores": [round(score, 4) for score in logits.tolist()] if logits is not None else [],
                "labels": phrases if phrases is not None else []
            },
            "execution_time": execution_time
        }
    
    @staticmethod
    def extractinfo_analysis(image_base64: str, list_prompt: list = []) -> Dict:
        '''Gera captions usando Florence-2 a partir de imagem em base64'''
        global florence_model, florence_processor, device

        start_time = time.time()

        # Convert base64 to PIL Image
        image = Image.open(io.BytesIO(base64.b64decode(image_base64)))
        img_np = np.array(image)

        prompts = [
            '<CAPTION_TO_PHRASE_GROUNDING>',
            '<CAPTION>',
            '<DETAILED_CAPTION>', 
            '<MORE_DETAILED_CAPTION>',
            '<DENSE_REGION_CAPTION>',
            '<OD>',
        ]

        results = {
            'execution_times': {},
            'total_time': 0,
            'annotated_images': {}
        }
        
        for prompt in prompts:
            prompt_start = time.time()
            
            prompt_text = prompt if prompt != '<CAPTION_TO_PHRASE_GROUNDING>' else prompt+' '.join([f'{x}.' for x in list_prompt])
            inputs = florence_processor(text=prompt_text, images=image, return_tensors='pt').to(device)
            
            generated_ids = florence_model.generate(
                input_ids=inputs['input_ids'],
                pixel_values=inputs['pixel_values'],
                max_new_tokens=1024,
                do_sample=False,
                num_beams=3,
                early_stopping=False,
                use_cache=False,
            )
            generated_text = florence_processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
            parsed_answer = florence_processor.post_process_generation(
                generated_text,
                task=prompt,
                image_size=(image.width, image.height)
            )
            
            prompt_time = time.time() - prompt_start
            results[prompt] = parsed_answer.get(prompt, '')
            results['execution_times'][prompt] = round(prompt_time, 2)
        
        results['total_time'] = round(time.time() - start_time, 2)

        def add_annotations(img_np, bboxes, labels, color):
            fig, ax = plt.subplots(figsize=(10, 10))
            ax.imshow(img_np)
            for bbox, label in zip(bboxes, labels):
                x, y, w, h = bbox
                rect = plt.Rectangle((x, y), w-x, h-y, linewidth=1, 
                                    edgecolor=color, facecolor='none')
                ax.add_patch(rect)
                ax.text(x, y, label, color='white', backgroundcolor=color)
            plt.axis('off')
            buf = io.BytesIO()
            plt.savefig(buf, format='jpg', bbox_inches='tight', pad_inches=0)
            plt.close()
            buf.seek(0)
            return base64.b64encode(buf.getvalue()).decode('utf-8')

        annotated_images = {}

        if '<DENSE_REGION_CAPTION>' in results and 'bboxes' in results['<DENSE_REGION_CAPTION>']:
            annotated_images['dense_region_caption'] = add_annotations(
                img_np, 
                results['<DENSE_REGION_CAPTION>']['bboxes'],
                results['<DENSE_REGION_CAPTION>']['labels'],
                '#a83232'
            )

        if '<OD>' in results and 'bboxes' in results['<OD>']:
            annotated_images['od'] = add_annotations(
                img_np,
                results['<OD>']['bboxes'],
                results['<OD>']['labels'], 
                '#3251a8'
            )

        if '<CAPTION_TO_PHRASE_GROUNDING>' in results and 'bboxes' in results['<CAPTION_TO_PHRASE_GROUNDING>']:
            annotated_images['phrase_grounding'] = add_annotations(
                img_np,
                results['<CAPTION_TO_PHRASE_GROUNDING>']['bboxes'],
                results['<CAPTION_TO_PHRASE_GROUNDING>']['labels'],
                '#5f32a8'
            )

        results['annotated_images'] = annotated_images
        return results

In [8]:
class MonitoringSystem:
    """Sistema principal de monitoramento"""
    
    def __init__(self):
        self.db_managers = {}
    
    def get_database_manager(self, company_id: str) -> DatabaseManager:
        """Obtém ou cria gerenciador de banco para a empresa"""
        if company_id not in self.db_managers:
            self.db_managers[company_id] = DatabaseManager(company_id)
        return self.db_managers[company_id]
    
    def process_incoming_data(self, data: dict) -> dict:
        """Processa dados recebidos do coletor"""
        try:
            # Extrair informações básicas
            company_info = data['company_info']
            company_id = company_info['id']
            collector_info = company_info['collector']
            collector_id = collector_info['id']
            collector_model = collector_info['model']
            timestamp = data['system_metadata']['timestamp']
            
            # Criar estrutura de diretórios
            collector_path = DirectoryManager.create_company_structure(
                company_id, collector_id
            )
            
            # Obter gerenciador de banco
            db_manager = self.get_database_manager(company_id)
            
            # Inserir dados da empresa e coletor
            db_manager.insert_company(company_id, company_info['name'])
            
            collector_data = {
                'id': collector_id,
                'model': collector_model,
                'description': collector_info.get('description', ''),
                'latitude': company_info['fixed_location']['latitude'],
                'longitude': company_info['fixed_location']['longitude']
            }
            db_manager.insert_collector(collector_data, company_id)
            
            # Processar e salvar imagens originais
            image_data = data['image_data']
            
            # Salvar imagem original (raw)
            raw_filename = DirectoryManager.generate_filename(
                "RAW", company_id, collector_id, collector_model, timestamp
            )
            raw_path = collector_path / "raw_capture" / raw_filename
            DirectoryManager.save_base64_image(
                image_data['original']['base64'], str(raw_path)
            )
            
            # Salvar imagem processada (pyl)
            pyl_filename = DirectoryManager.generate_filename(
                "PYL", company_id, collector_id, collector_model, timestamp
            )
            pyl_path = collector_path / "pyl_model" / pyl_filename
            DirectoryManager.save_base64_image(
                image_data['processed']['base64'], str(pyl_path)
            )
            
            # Inserir dados da captura
            capture_data = {
                'company_id': company_id,
                'collector_id': collector_id,
                'timestamp': timestamp,
                'raw_image_path': str(raw_path),
                'processed_image_path': str(pyl_path),
                'raw_image_size_mb': image_data['original']['size_mb'],
                'processed_image_size_mb': image_data['processed']['size_mb'],
                'camera_settings': data['system_metadata']['camera'],
                'system_metadata': data['system_metadata'],
                'system_status': data['system_status']
            }
            capture_id = db_manager.insert_capture(capture_data)
            
            # Inserir detecções originais
            db_manager.insert_original_detections(
                capture_id, data['detections']['data']
            )
            
            # Executar análise Grounding DINO
            seek_data = data.get('seek_on_image', {})
            if seek_data and 'user_prompt' in seek_data:
                grounding_result = self.run_grounding_dino_analysis(
                    image_data['original']['base64'],
                    seek_data['user_prompt'],
                    seek_data.get('prompt_title', ''),
                    collector_path,
                    company_id, collector_id, collector_model, timestamp
                )
                
                if grounding_result:
                    analysis_id = db_manager.insert_grounding_dino_analysis(
                        capture_id, grounding_result
                    )
                    db_manager.insert_grounding_dino_detections(
                        analysis_id, grounding_result['detections']
                    )
            
            # Executar análise Florence-2
            florence_result = self.run_florence2_analysis(
                image_data['original']['base64'],
                collector_path,
                company_id, collector_id, collector_model, timestamp
            )
            
            if florence_result:
                analysis_id = db_manager.insert_florence2_analysis(
                    capture_id, florence_result
                )
                db_manager.insert_florence2_execution_times(
                    analysis_id, florence_result['execution_times']
                )
                db_manager.insert_florence2_detections(
                    analysis_id, florence_result
                )
            
            return {
                'status': 'success',
                'message': 'Dados processados com sucesso',
                'capture_id': capture_id,
                'company_id': company_id,
                'collector_id': collector_id
            }
            
        except Exception as e:
            logger.error(f"Erro ao processar dados: {str(e)}")
            return {
                'status': 'error',
                'message': f'Erro ao processar dados: {str(e)}'
            }
    
    def run_grounding_dino_analysis(self, image_base64: str, user_prompts: list,
                                   prompt_title: str, collector_path: Path,
                                   company_id: str, collector_id: str, 
                                   model: str, timestamp: str) -> dict:
        """Executa análise Grounding DINO"""
        try:
            # Executar análise (substituir pela sua função)
            result = ModelAnalyzer.seeknow_analysis(
                image_base64, user_prompts, prompt_title
            )
            
            if result and 'image' in result:
                # Salvar imagem resultado
                grd_filename = DirectoryManager.generate_filename(
                    "GRD", company_id, collector_id, model, timestamp
                )
                grd_path = collector_path / "grd_model" / grd_filename
                DirectoryManager.save_base64_image(
                    result['image'], str(grd_path)
                )
                
                # Adicionar caminho da imagem ao resultado
                result['image_path'] = str(grd_path)
                result['user_prompts'] = user_prompts
                
                return result
                
        except Exception as e:
            logger.error(f"Erro na análise Grounding DINO: {str(e)}")
            
        return None
    
    def run_florence2_analysis(self, image_base64: str, collector_path: Path,
                              company_id: str, collector_id: str, 
                              model: str, timestamp: str) -> dict:
        """Executa análise Florence-2"""
        try:
            # Executar análise (substituir pela sua função)
            result = ModelAnalyzer.extractinfo_analysis(image_base64)
            
            if result and 'annotated_images' in result:
                image_paths = {}
                
                # Salvar imagem dense_region_caption
                if 'dense_region_caption' in result['annotated_images']:
                    fld_filename = DirectoryManager.generate_filename(
                        "FLD", company_id, collector_id, model, timestamp
                    )
                    fld_path = collector_path / "fld_model" / fld_filename
                    DirectoryManager.save_base64_image(
                        result['annotated_images']['dense_region_caption'], 
                        str(fld_path)
                    )
                    image_paths['dense_region_caption'] = str(fld_path)
                
                # Salvar imagem od
                if 'od' in result['annotated_images']:
                    flo_filename = DirectoryManager.generate_filename(
                        "FLO", company_id, collector_id, model, timestamp
                    )
                    flo_path = collector_path / "flo_model" / flo_filename
                    DirectoryManager.save_base64_image(
                        result['annotated_images']['od'], str(flo_path)
                    )
                    image_paths['od'] = str(flo_path)
                
                # Salvar imagem phrase_grounding
                if 'phrase_grounding' in result['annotated_images']:
                    flp_filename = DirectoryManager.generate_filename(
                        "FLP", company_id, collector_id, model, timestamp
                    )
                    flp_path = collector_path / "flp_model" / flp_filename
                    DirectoryManager.save_base64_image(
                        result['annotated_images']['phrase_grounding'], 
                        str(flp_path)
                    )
                    image_paths['phrase_grounding'] = str(flp_path)
                
                # Adicionar caminhos das imagens ao resultado
                result['image_paths'] = image_paths
                
                return result
                
        except Exception as e:
            logger.error(f"Erro na análise Florence-2: {str(e)}")
            
        return None

In [9]:
monitoring_system = MonitoringSystem()

def background_processor():
    """Processa dados em background"""
    while True:
        try:
            data = processing_queue.get()
            if data is None:  # Sinal para parar
                break
            
            # Fazer o processamento pesado aqui
            result = monitoring_system.process_incoming_data(data)
            logger.info(f"Background processing completed: {result.get('status', 'unknown')}")
            
        except Exception as e:
            logger.error(f"Error in background processing: {e}")
        finally:
            processing_queue.task_done()

background_thread = threading.Thread(target=background_processor, daemon=True)
background_thread.start()

# Rotas Flask
@app.route('/upload', methods=['POST'])
def upload_data():
    """Endpoint com resposta imediata"""
    try:
        data = request.get_json()
        
        if not data:
            return jsonify({
                'status': 'error',
                'message': 'Nenhum dado recebido'
            }), 400
        
        # VALIDAÇÃO RÁPIDA apenas
        required_fields = ['company_info', 'detections', 'image_data']
        if not all(field in data for field in required_fields):
            return jsonify({
                'status': 'error',
                'message': 'Dados incompletos'
            }), 400
        
        # Adicionar à fila para processamento background
        processing_queue.put(data)
        
        # RESPOSTA IMEDIATA (< 1 segundo)
        return jsonify({
            'status': 'accepted',
            'message': 'Dados recebidos e em processamento',
            'queue_size': processing_queue.qsize(),
        }), 202  # 202 = Accepted for processing
        
    except Exception as e:
        logger.error(f"Erro no endpoint /upload: {str(e)}")
        return jsonify({
            'status': 'error',
            'message': f'Erro interno: {str(e)}'
        }), 500

@app.route('/health', methods=['GET'])
def health_check():
    """Endpoint para verificar saúde do sistema"""
    return jsonify({
        'status': 'healthy',
        'base_data_dir': BASE_DATA_DIR
    })

@app.route('/stats/<company_id>', methods=['GET'])
def get_company_stats(company_id: str):
    """Endpoint para obter estatísticas de uma empresa"""
    try:
        db_manager = monitoring_system.get_database_manager(company_id)
        
        with sqlite3.connect(db_manager.db_path) as conn:
            cursor = conn.cursor()
            
            # Contar capturas
            cursor.execute('SELECT COUNT(*) FROM captures WHERE company_id = ?', 
                          (company_id,))
            capture_count = cursor.fetchone()[0]
            
            # Contar coletores
            cursor.execute('SELECT COUNT(*) FROM collectors WHERE company_id = ?', 
                          (company_id,))
            collector_count = cursor.fetchone()[0]
            
            # Última captura
            cursor.execute('''
                SELECT timestamp FROM captures 
                WHERE company_id = ? 
                ORDER BY created_at DESC LIMIT 1
            ''', (company_id,))
            last_capture = cursor.fetchone()
            
            return jsonify({
                'company_id': company_id,
                'capture_count': capture_count,
                'collector_count': collector_count,
                'last_capture': last_capture[0] if last_capture else None
            })
            
    except Exception as e:
        logger.error(f"Erro ao obter estatísticas: {str(e)}")
        return jsonify({
            'status': 'error',
            'message': f'Erro ao obter estatísticas: {str(e)}'
        }), 500

# Rotas da Interface Web
@app.route('/')
def dashboard():
    """Dashboard principal com lista de empresas"""
    try:
        companies = []
        
        if os.path.exists(BASE_DATA_DIR):
            for company_dir in os.listdir(BASE_DATA_DIR):
                company_path = os.path.join(BASE_DATA_DIR, company_dir)
                if os.path.isdir(company_path):
                    db_path = os.path.join(company_path, f"{company_dir}_database.db")
                    if os.path.exists(db_path):
                        db_manager = monitoring_system.get_database_manager(company_dir)
                        with sqlite3.connect(db_manager.db_path) as conn:
                            cursor = conn.cursor()
                            
                            # Obter dados da empresa
                            cursor.execute('SELECT name FROM companies WHERE id = ?', (company_dir,))
                            company_name = cursor.fetchone()
                            
                            # Estatísticas
                            cursor.execute('SELECT COUNT(*) FROM captures WHERE company_id = ?', (company_dir,))
                            captures = cursor.fetchone()[0]
                            
                            cursor.execute('SELECT COUNT(*) FROM collectors WHERE company_id = ?', (company_dir,))
                            collectors = cursor.fetchone()[0]
                            
                            cursor.execute('SELECT COUNT(*) FROM grounding_dino_analysis WHERE capture_id IN (SELECT id FROM captures WHERE company_id = ?)', (company_dir,))
                            grd_analyses = cursor.fetchone()[0]
                            
                            cursor.execute('SELECT COUNT(*) FROM florence2_analysis WHERE capture_id IN (SELECT id FROM captures WHERE company_id = ?)', (company_dir,))
                            florence_analyses = cursor.fetchone()[0]
                            
                            companies.append({
                                'id': company_dir,
                                'name': company_name[0] if company_name else company_dir,
                                'captures': captures,
                                'collectors': collectors,
                                'grd_analyses': grd_analyses,
                                'florence_analyses': florence_analyses
                            })
        
        # Renderizar o HTML completo
        if companies:
            companies_html = ''.join([f'''
                <div class="company-card" onclick="window.location.href='/company/{company['id']}'">
                    <div class="company-header">
                        <div class="company-icon">🏢</div>
                        <div class="company-info">
                            <h3>{company['name']}</h3>
                            <p>ID: {company['id']}</p>
                        </div>
                    </div>
                    <div class="stats-grid">
                        <div class="stat-item">
                            <span class="stat-number">{company['captures']}</span>
                            <div class="stat-label">Capturas</div>
                        </div>
                        <div class="stat-item">
                            <span class="stat-number">{company['collectors']}</span>
                            <div class="stat-label">Coletores</div>
                        </div>
                        <div class="stat-item">
                            <span class="stat-number">{company['grd_analyses']}</span>
                            <div class="stat-label">Análises DINO</div>
                        </div>
                        <div class="stat-item">
                            <span class="stat-number">{company['florence_analyses']}</span>
                            <div class="stat-label">Análises Florence</div>
                        </div>
                    </div>
                </div>
            ''' for company in companies])
        else:
            companies_html = '''
                <div class="empty-state">
                    <h2>📊 Nenhuma empresa encontrada</h2>
                    <p>Envie dados através do endpoint /upload para começar a ver informações aqui</p>
                </div>
            '''
        
        html = f'''
        <!DOCTYPE html>
        <html lang="pt-BR">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Sistema de Monitoramento - Dashboard</title>
            <style>
                * {{ margin: 0; padding: 0; box-sizing: border-box; }}
                body {{ 
                    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    min-height: 100vh;
                    color: #333;
                }}
                .header {{
                    background: rgba(255,255,255,0.95);
                    backdrop-filter: blur(10px);
                    padding: 20px 0;
                    box-shadow: 0 4px 20px rgba(0,0,0,0.1);
                }}
                .container {{ max-width: 1200px; margin: 0 auto; padding: 0 20px; }}
                .header h1 {{
                    text-align: center;
                    color: #4a5568;
                    font-size: 2.5rem;
                    margin-bottom: 10px;
                }}
                .header p {{
                    text-align: center;
                    color: #718096;
                    font-size: 1.1rem;
                }}
                .main-content {{ padding: 40px 20px; }}
                .companies-grid {{
                    display: grid;
                    grid-template-columns: repeat(auto-fit, minmax(350px, 1fr));
                    gap: 30px;
                    max-width: 1200px;
                    margin: 0 auto;
                }}
                .company-card {{
                    background: rgba(255,255,255,0.95);
                    backdrop-filter: blur(10px);
                    border-radius: 20px;
                    padding: 30px;
                    box-shadow: 0 8px 40px rgba(0,0,0,0.1);
                    transition: all 0.3s ease;
                    cursor: pointer;
                    border: 2px solid transparent;
                }}
                .company-card:hover {{
                    transform: translateY(-10px);
                    box-shadow: 0 20px 60px rgba(0,0,0,0.15);
                    border-color: #667eea;
                }}
                .company-header {{
                    display: flex;
                    align-items: center;
                    margin-bottom: 20px;
                }}
                .company-icon {{
                    width: 60px;
                    height: 60px;
                    background: linear-gradient(45deg, #667eea, #764ba2);
                    border-radius: 15px;
                    display: flex;
                    align-items: center;
                    justify-content: center;
                    margin-right: 15px;
                    font-size: 24px;
                    color: white;
                }}
                .company-info h3 {{
                    font-size: 1.4rem;
                    color: #2d3748;
                    margin-bottom: 5px;
                }}
                .company-info p {{
                    color: #718096;
                    font-size: 0.9rem;
                }}
                .stats-grid {{
                    display: grid;
                    grid-template-columns: 1fr 1fr;
                    gap: 15px;
                    margin-top: 20px;
                }}
                .stat-item {{
                    text-align: center;
                    padding: 15px;
                    background: linear-gradient(45deg, #f7fafc, #edf2f7);
                    border-radius: 12px;
                }}
                .stat-number {{
                    font-size: 1.8rem;
                    font-weight: bold;
                    color: #4a5568;
                    display: block;
                }}
                .stat-label {{
                    font-size: 0.85rem;
                    color: #718096;
                    text-transform: uppercase;
                    letter-spacing: 0.5px;
                    margin-top: 5px;
                }}
                .empty-state {{
                    text-align: center;
                    padding: 60px 20px;
                }}
                .empty-state h2 {{
                    color: rgba(255,255,255,0.9);
                    font-size: 2rem;
                    margin-bottom: 15px;
                }}
                .empty-state p {{
                    color: rgba(255,255,255,0.7);
                    font-size: 1.1rem;
                }}
                @media (max-width: 768px) {{
                    .header h1 {{ font-size: 2rem; }}
                    .companies-grid {{ grid-template-columns: 1fr; }}
                    .company-card {{ padding: 20px; }}
                }}
            </style>
        </head>
        <body>
            <div class="header">
                <div class="container">
                    <h1>🚀 Sistema de Monitoramento</h1>
                    <p>Dashboard Executivo - Visão Geral das Empresas</p>
                </div>
            </div>
            
            <div class="main-content">
                <div class="companies-grid">
                    {companies_html}
                </div>
            </div>
        </body>
        </html>
        '''
        return html
        
    except Exception as e:
        logger.error(f"Erro no dashboard: {str(e)}")
        return f"<h1>Erro: {str(e)}</h1>", 500

@app.route('/capture/<company_id>/<int:capture_id>')
def capture_details(company_id, capture_id):
    """Detalhes completos de uma captura específica"""
    try:
        db_manager = monitoring_system.get_database_manager(company_id)
        
        with sqlite3.connect(db_manager.db_path) as conn:
            cursor = conn.cursor()
            
            # Modified query with correct column names
            cursor.execute('''
                SELECT 
                    c.id,
                    c.collector_id,
                    c.timestamp,
                    c.raw_image_path,
                    c.processed_image_path,
                    c.raw_image_size_mb,
                    c.processed_image_size_mb,
                    c.camera_settings,
                    c.system_metadata,
                    c.system_status,
                    co.model,
                    co.description
                FROM captures c
                JOIN collectors co ON c.collector_id = co.id
                WHERE c.id = ? AND c.company_id = ?
            ''', (capture_id, company_id))
            capture_info = cursor.fetchone()
            
            if not capture_info:
                return "<h1>Captura não encontrada</h1>", 404
            
            # Create a dictionary with correct field names
            capture_data = {
                'id': capture_info[0],
                'collector_id': capture_info[1],
                'timestamp': capture_info[2],
                'raw_image_path': capture_info[3],
                'processed_image_path': capture_info[4],
                'raw_image_size_mb': capture_info[5],
                'processed_image_size_mb': capture_info[6],
                'camera_settings': capture_info[7],
                'system_metadata': capture_info[8],
                'system_status': capture_info[9],
                'model': capture_info[10],
                'description': capture_info[11]
            }

            # Detecções originais
            cursor.execute('''
                SELECT x, y, width, height, confidence, class_name, class_id
                FROM original_detections
                WHERE capture_id = ?
                ORDER BY confidence DESC
            ''', (capture_id,))
            original_detections = cursor.fetchall()
            
            # Análise Grounding DINO
            cursor.execute('''
                SELECT gda.*, gdd.x, gdd.y, gdd.width, gdd.height, gdd.score, gdd.label
                FROM grounding_dino_analysis gda
                LEFT JOIN grounding_dino_detections gdd ON gda.id = gdd.analysis_id
                WHERE gda.capture_id = ?
                ORDER BY gdd.score DESC
            ''', (capture_id,))
            grounding_results = cursor.fetchall()
            
            # Análise Florence-2
            cursor.execute('''
                SELECT fa.*, fet.task_name, fet.execution_time
                FROM florence2_analysis fa
                LEFT JOIN florence2_execution_times fet ON fa.id = fet.analysis_id
                WHERE fa.capture_id = ?
            ''', (capture_id,))
            florence_results = cursor.fetchall()
            
            # Detecções Florence-2
            cursor.execute('''
                SELECT 'phrase' as type, x1, y1, x2, y2, label
                FROM florence2_phrase_detections
                WHERE analysis_id IN (SELECT id FROM florence2_analysis WHERE capture_id = ?)
                UNION ALL
                SELECT 'dense' as type, x1, y1, x2, y2, label
                FROM florence2_dense_regions
                WHERE analysis_id IN (SELECT id FROM florence2_analysis WHERE capture_id = ?)
                UNION ALL
                SELECT 'object' as type, x1, y1, x2, y2, label
                FROM florence2_object_detections
                WHERE analysis_id IN (SELECT id FROM florence2_analysis WHERE capture_id = ?)
            ''', (capture_id, capture_id, capture_id))
            florence_detections = cursor.fetchall()
            
            # Parse dos metadados JSON
            import json
            def safe_json_parse(value):
                if isinstance(value, (str, bytes)):
                    try:
                        return json.loads(value) if value else {}
                    except json.JSONDecodeError:
                        return {}
                elif isinstance(value, dict):
                    return value
                return {}

            camera_settings = safe_json_parse(capture_data['camera_settings'])
            system_metadata = safe_json_parse(capture_data['system_metadata'])
            system_status = safe_json_parse(capture_data['system_status'])
        
        html = f'''
        <!DOCTYPE html>
        <html lang="pt-BR">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Captura #{capture_id} - Detalhes</title>
            <style>
                * {{ margin: 0; padding: 0; box-sizing: border-box; }}
                body {{ 
                    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    min-height: 100vh;
                    color: #333;
                }}
                .header {{
                    background: rgba(255,255,255,0.95);
                    backdrop-filter: blur(10px);
                    padding: 20px 0;
                    box-shadow: 0 4px 20px rgba(0,0,0,0.1);
                    position: sticky;
                    top: 0;
                    z-index: 100;
                }}
                .container {{ max-width: 1400px; margin: 0 auto; padding: 0 20px; }}
                .header-content {{
                    display: flex;
                    justify-content: space-between;
                    align-items: center;
                }}
                .back-btn {{
                    background: linear-gradient(45deg, #667eea, #764ba2);
                    color: white;
                    padding: 10px 20px;
                    border: none;
                    border-radius: 25px;
                    cursor: pointer;
                    text-decoration: none;
                    transition: all 0.3s ease;
                }}
                .main-content {{ padding: 30px 20px; }}
                .section {{
                    background: rgba(255,255,255,0.95);
                    backdrop-filter: blur(10px);
                    border-radius: 20px;
                    padding: 30px;
                    margin-bottom: 30px;
                    box-shadow: 0 8px 40px rgba(0,0,0,0.1);
                }}
                .section h2 {{
                    color: #4a5568;
                    margin-bottom: 20px;
                    font-size: 1.8rem;
                    display: flex;
                    align-items: center;
                    gap: 10px;
                }}
                .info-grid {{
                    display: grid;
                    grid-template-columns: repeat(auto-fit, minmax(300px, 1fr));
                    gap: 20px;
                    margin-bottom: 20px;
                }}
                .info-card {{
                    background: linear-gradient(45deg, #f7fafc, #edf2f7);
                    border-radius: 15px;
                    padding: 20px;
                }}
                .info-card h3 {{
                    color: #4a5568;
                    margin-bottom: 15px;
                    font-size: 1.2rem;
                }}
                .info-item {{
                    display: flex;
                    justify-content: space-between;
                    margin-bottom: 8px;
                    padding: 5px 0;
                    border-bottom: 1px solid rgba(0,0,0,0.05);
                }}
                .info-item:last-child {{ border-bottom: none; }}
                .detections-grid {{
                    display: grid;
                    grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
                    gap: 15px;
                }}
                .detection-card {{
                    background: linear-gradient(45deg, #f7fafc, #edf2f7);
                    border-radius: 12px;
                    padding: 15px;
                    border-left: 4px solid;
                }}
                .detection-original {{ border-left-color: #48bb78; }}
                .detection-grounding {{ border-left-color: #ed8936; }}
                .detection-florence {{ border-left-color: #9f7aea; }}
                .confidence-bar {{
                    width: 100%;
                    height: 8px;
                    background: #e2e8f0;
                    border-radius: 4px;
                    overflow: hidden;
                    margin-top: 10px;
                }}
                .confidence-fill {{
                    height: 100%;
                    background: linear-gradient(45deg, #667eea, #764ba2);
                    transition: width 0.3s ease;
                }}
                .tabs {{
                    display: flex;
                    background: linear-gradient(45deg, #f7fafc, #edf2f7);
                    border-radius: 15px;
                    padding: 5px;
                    margin-bottom: 20px;
                }}
                .tab-btn {{
                    flex: 1;
                    padding: 12px 20px;
                    border: none;
                    background: none;
                    border-radius: 10px;
                    cursor: pointer;
                    transition: all 0.3s ease;
                    font-weight: 500;
                }}
                .tab-btn.active {{
                    background: linear-gradient(45deg, #667eea, #764ba2);
                    color: white;
                }}
                .tab-content {{ display: none; }}
                .tab-content.active {{ display: block; }}
            </style>
        </head>
        <body>
            <div class="header">
                <div class="container">
                    <div class="header-content">
                        <a href="/collector/{company_id}/{capture_info[2]}" class="back-btn">⬅ Voltar</a>
                        <div>
                            <h1>📸 Captura #{capture_id}</h1>
                            <p>Coletor: {capture_info[2]} | {capture_info[3]}</p>
                        </div>
                        <div></div>
                    </div>
                </div>
            </div>
            
            <div class="main-content">
                <div class="container">
                    <div class="section">
                        <h2>ℹ️ Informações Gerais</h2>
                        <div class="info-grid">
                            <div class="info-card">
                                <h3>📊 Dados da Captura</h3>
                                <div class="info-item">
                                    <span><strong>Timestamp:</strong></span>
                                    <span>{capture_info[3]}</span>
                                </div>
                                <div class="info-item">
                                    <span><strong>Coletor:</strong></span>
                                    <span>{capture_info[2]}</span>
                                </div>
                                <div class="info-item">
                                    <span><strong>Modelo:</strong></span>
                                    <span>{capture_info[11]}</span>
                                </div>
                                <div class="info-item">
                                    <span><strong>Tamanho RAW:</strong></span>
                                    <span>{capture_info[5]} MB</span>
                                </div>
                                <div class="info-item">
                                    <span><strong>Tamanho Processado:</strong></span>
                                    <span>{capture_info[6]} MB</span>
                                </div>
                            </div>
                            
                            <div class="info-card">
                                <h3>📷 Configurações da Câmera</h3>
                                <div class="info-item">
                                    <span><strong>Resolução:</strong></span>
                                    <span>{camera_settings.get('resolution', {}).get('width', 'N/A')}x{camera_settings.get('resolution', {}).get('height', 'N/A')}</span>
                                </div>
                                <div class="info-item">
                                    <span><strong>FPS:</strong></span>
                                    <span>{camera_settings.get('settings', {}).get('fps', 'N/A')}</span>
                                </div>
                                <div class="info-item">
                                    <span><strong>Brilho:</strong></span>
                                    <span>{camera_settings.get('settings', {}).get('brightness', 'N/A')}</span>
                                </div>
                                <div class="info-item">
                                    <span><strong>Contraste:</strong></span>
                                    <span>{camera_settings.get('settings', {}).get('contrast', 'N/A')}</span>
                                </div>
                                <div class="info-item">
                                    <span><strong>Exposição:</strong></span>
                                    <span>{camera_settings.get('settings', {}).get('exposure', 'N/A')}</span>
                                </div>
                            </div>
                            
                            <div class="info-card">
                                <h3>💻 Status do Sistema</h3>
                                <div class="info-item">
                                    <span><strong>CPU:</strong></span>
                                    <span>{system_status.get('cpu', 'N/A')}%</span>
                                </div>
                                <div class="info-item">
                                    <span><strong>Memória:</strong></span>
                                    <span>{system_status.get('memory', 'N/A')}%</span>
                                </div>
                                <div class="info-item">
                                    <span><strong>Uptime:</strong></span>
                                    <span>{system_status.get('uptime', 'N/A')}s</span>
                                </div>
                                <div class="info-item">
                                    <span><strong>SO:</strong></span>
                                    <span>{system_metadata.get('os', {}).get('system', 'N/A')}</span>
                                </div>
                                <div class="info-item">
                                    <span><strong>Arquitetura:</strong></span>
                                    <span>{system_metadata.get('hardware', {}).get('architecture', 'N/A')}</span>
                                </div>
                            </div>
                        </div>
                    </div>
                    
                    <div class="section">
                        <h2>🎯 Análise de Detecções</h2>
                        <div class="tabs">
                            <button class="tab-btn active" onclick="showTab('original')">Detecções Originais ({len(original_detections)})</button>
                            <button class="tab-btn" onclick="showTab('grounding')">Grounding DINO ({len([r for r in grounding_results if r[4] is not None])})</button>
                            <button class="tab-btn" onclick="showTab('florence')">Florence-2 ({len(florence_detections)})</button>
                        </div>
                        
                        <div id="original" class="tab-content active">
                            <div class="detections-grid">
                                {''.join([f'''
                                <div class="detection-card detection-original">
                                    <h4>{detection[5]}</h4>
                                    <p><strong>Posição:</strong> ({detection[0]}, {detection[1]})</p>
                                    <p><strong>Tamanho:</strong> {detection[2]}x{detection[3]}</p>
                                    <p><strong>Class ID:</strong> {detection[6]}</p>
                                    <div class="confidence-bar">
                                        <div class="confidence-fill" style="width: {detection[4]*100}%"></div>
                                    </div>
                                    <p style="text-align: center; margin-top: 5px; font-weight: bold;">Confiança: {detection[4]:.2%}</p>
                                </div>
                                ''' for detection in original_detections])}
                            </div>
                        </div>
                        
                        <div id="grounding" class="tab-content">
                            <div class="detections-grid">
                                {''.join([f'''
                                <div class="detection-card detection-grounding">
                                    <h4>{result[10]}</h4>
                                    <p><strong>Posição:</strong> ({result[6]}, {result[7]})</p>
                                    <p><strong>Tamanho:</strong> {result[8]}x{result[9]}</p>
                                    <div class="confidence-bar">
                                        <div class="confidence-fill" style="width: {result[11]*100}%"></div>
                                    </div>
                                    <p style="text-align: center; margin-top: 5px; font-weight: bold;">Score: {result[11]:.2%}</p>
                                </div>
                                ''' for result in grounding_results if result[4] is not None])}
                            </div>
                            {f'<p><strong>Tempo de Execução:</strong> {grounding_results[0][5]:.2f}s</p>' if grounding_results and grounding_results[0][5] else ''}
                        </div>
                        
                        <div id="florence" class="tab-content">
                            <div class="detections-grid">
                                {''.join([f'''
                                <div class="detection-card detection-florence">
                                    <h4>{detection[5]} ({detection[0].upper()})</h4>
                                    <p><strong>Coordenadas:</strong> ({detection[1]:.1f}, {detection[2]:.1f}) → ({detection[3]:.1f}, {detection[4]:.1f})</p>
                                    <p><strong>Tipo:</strong> {{'phrase': 'Phrase Grounding', 'dense': 'Dense Region', 'object': 'Object Detection'}}[detection[0]]</p>
                                </div>
                                ''' for detection in florence_detections])}
                            </div>
                            {f'<div style="margin-top: 20px;"><h4>Tempos de Execução Florence-2:</h4><ul>{"".join([f"<li><strong>{result[11]}:</strong> {result[12]:.2f}s</li>" for result in florence_results if result[11]])}</ul></div>' if florence_results else ''}
                        </div>
                    </div>
                    
                    <div class="section">
                        <h2>🖼️ Imagens Processadas</h2>
                        <div class="info-grid">
                            <div class="info-card">
                                <h3>📁 Imagem Original (RAW)</h3>
                                <p><strong>Arquivo:</strong> {os.path.basename(capture_info[4])}</p>
                                <p><strong>Caminho:</strong> {capture_info[4]}</p>
                            </div>
                            <div class="info-card">
                                <h3>🎨 Imagem Processada</h3>
                                <p><strong>Arquivo:</strong> {os.path.basename(capture_info[5])}</p>
                                <p><strong>Caminho:</strong> {capture_info[5]}</p>
                            </div>
                        </div>
                    </div>
                </div>
            </div>
            
            <script>
                function showTab(tabName) {{
                    // Ocultar todas as abas
                    document.querySelectorAll('.tab-content').forEach(tab => {{
                        tab.classList.remove('active');
                    }});
                    document.querySelectorAll('.tab-btn').forEach(btn => {{
                        btn.classList.remove('active');
                    }});
                    
                    // Mostrar aba selecionada
                    document.getElementById(tabName).classList.add('active');
                    event.target.classList.add('active');
                }}
            </script>
        </body>
        </html>
        '''
        return html
        
    except Exception as e:
        logger.error(f"Erro nos detalhes da captura: {str(e)}")
        return f"<h1>Erro: {str(e)}</h1>", 500

@app.route('/api/companies')
def api_companies():
    """API endpoint para listar empresas"""
    try:
        companies = []
        if os.path.exists(BASE_DATA_DIR):
            for company_dir in os.listdir(BASE_DATA_DIR):
                company_path = os.path.join(BASE_DATA_DIR, company_dir)
                if os.path.isdir(company_path):
                    db_path = os.path.join(company_path, f"{company_dir}_database.db")
                    if os.path.exists(db_path):
                        companies.append(company_dir)
        
        return jsonify({
            'status': 'success',
            'companies': companies,
            'count': len(companies)
        })
        
    except Exception as e:
        return jsonify({
            'status': 'error',
            'message': str(e)
        }), 500

@app.route('/api/company/<company_id>/stats')
def api_company_stats(company_id):
    """API endpoint para estatísticas de uma empresa"""
    try:
        db_manager = monitoring_system.get_database_manager(company_id)
        
        with sqlite3.connect(db_manager.db_path) as conn:
            cursor = conn.cursor()
            
            # Estatísticas básicas
            cursor.execute('SELECT COUNT(*) FROM captures WHERE company_id = ?', (company_id,))
            captures = cursor.fetchone()[0]
            
            cursor.execute('SELECT COUNT(*) FROM collectors WHERE company_id = ?', (company_id,))
            collectors = cursor.fetchone()[0]
            
            cursor.execute('SELECT COUNT(*) FROM original_detections WHERE capture_id IN (SELECT id FROM captures WHERE company_id = ?)', (company_id,))
            detections = cursor.fetchone()[0]
            
            # Detecções por classe (top 10)
            cursor.execute('''
                SELECT class_name, COUNT(*) as count
                FROM original_detections 
                WHERE capture_id IN (SELECT id FROM captures WHERE company_id = ?)
                GROUP BY class_name
                ORDER BY count DESC
                LIMIT 10
            ''', (company_id,))
            top_classes = cursor.fetchall()
            
            # Capturas por dia (últimos 30 dias)
            cursor.execute('''
                SELECT DATE(timestamp) as date, COUNT(*) as count
                FROM captures 
                WHERE company_id = ? AND timestamp >= datetime('now', '-30 days')
                GROUP BY DATE(timestamp)
                ORDER BY date
            ''', (company_id,))
            daily_captures = cursor.fetchall()
            
            return jsonify({
                'status': 'success',
                'company_id': company_id,
                'stats': {
                    'captures': captures,
                    'collectors': collectors,
                    'detections': detections,
                    'top_classes': [{'class': row[0], 'count': row[1]} for row in top_classes],
                    'daily_captures': [{'date': row[0], 'count': row[1]} for row in daily_captures]
                }
            })
            
    except Exception as e:
        return jsonify({
            'status': 'error',
            'message': str(e)
        }), 500

@app.route('/company/<company_id>')
def company_dashboard(company_id):
    """Dashboard detalhado de uma empresa"""
    try:
        db_manager = monitoring_system.get_database_manager(company_id)
        
        with sqlite3.connect(db_manager.db_path) as conn:
            cursor = conn.cursor()
            
            # Informações da empresa
            cursor.execute('SELECT name FROM companies WHERE id = ?', (company_id,))
            company_result = cursor.fetchone()
            company_name = company_result[0] if company_result else company_id
            
            # Coletores
            cursor.execute('''
                SELECT id, model, description, latitude, longitude 
                FROM collectors WHERE company_id = ?
            ''', (company_id,))
            collectors = cursor.fetchall()
            
            # Últimas capturas
            cursor.execute('''
                SELECT c.id, c.collector_id, c.timestamp, c.raw_image_path, 
                       COUNT(od.id) as detections_count
                FROM captures c
                LEFT JOIN original_detections od ON c.id = od.capture_id
                WHERE c.company_id = ?
                GROUP BY c.id
                ORDER BY c.created_at DESC
                LIMIT 10
            ''', (company_id,))
            recent_captures = cursor.fetchall()
            
            # Estatísticas gerais
            cursor.execute('SELECT COUNT(*) FROM captures WHERE company_id = ?', (company_id,))
            total_captures = cursor.fetchone()[0]
            
            cursor.execute('SELECT COUNT(*) FROM original_detections WHERE capture_id IN (SELECT id FROM captures WHERE company_id = ?)', (company_id,))
            total_detections = cursor.fetchone()[0]
            
            cursor.execute('SELECT COUNT(*) FROM grounding_dino_analysis WHERE capture_id IN (SELECT id FROM captures WHERE company_id = ?)', (company_id,))
            total_grd = cursor.fetchone()[0]
            
            cursor.execute('SELECT COUNT(*) FROM florence2_analysis WHERE capture_id IN (SELECT id FROM captures WHERE company_id = ?)', (company_id,))
            total_florence = cursor.fetchone()[0]
        
        html = f'''
        <!DOCTYPE html>
        <html lang="pt-BR">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>{company_name} - Dashboard</title>
            <style>
                * {{ margin: 0; padding: 0; box-sizing: border-box; }}
                body {{ 
                    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    min-height: 100vh;
                    color: #333;
                }}
                .header {{
                    background: rgba(255,255,255,0.95);
                    backdrop-filter: blur(10px);
                    padding: 20px 0;
                    box-shadow: 0 4px 20px rgba(0,0,0,0.1);
                    position: sticky;
                    top: 0;
                    z-index: 100;
                }}
                .container {{ max-width: 1200px; margin: 0 auto; padding: 0 20px; }}
                .header-content {{
                    display: flex;
                    justify-content: space-between;
                    align-items: center;
                }}
                .back-btn {{
                    background: linear-gradient(45deg, #667eea, #764ba2);
                    color: white;
                    padding: 10px 20px;
                    border: none;
                    border-radius: 25px;
                    cursor: pointer;
                    text-decoration: none;
                    transition: all 0.3s ease;
                }}
                .back-btn:hover {{ transform: translateY(-2px); }}
                .company-title {{
                    text-align: center;
                    flex: 1;
                }}
                .company-title h1 {{
                    color: #4a5568;
                    font-size: 2.2rem;
                    margin-bottom: 5px;
                }}
                .company-title p {{ color: #718096; }}
                .main-content {{ padding: 30px 20px; }}
                .stats-overview {{
                    display: grid;
                    grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
                    gap: 20px;
                    margin-bottom: 40px;
                }}
                .stat-card {{
                    background: rgba(255,255,255,0.95);
                    backdrop-filter: blur(10px);
                    border-radius: 15px;
                    padding: 25px;
                    text-align: center;
                    box-shadow: 0 8px 30px rgba(0,0,0,0.1);
                }}
                .stat-card .icon {{
                    font-size: 2.5rem;
                    margin-bottom: 10px;
                }}
                .stat-card .number {{
                    font-size: 2.5rem;
                    font-weight: bold;
                    color: #4a5568;
                    display: block;
                }}
                .stat-card .label {{
                    color: #718096;
                    font-size: 0.9rem;
                    text-transform: uppercase;
                    letter-spacing: 0.5px;
                    margin-top: 5px;
                }}
                .section {{
                    background: rgba(255,255,255,0.95);
                    backdrop-filter: blur(10px);
                    border-radius: 20px;
                    padding: 30px;
                    margin-bottom: 30px;
                    box-shadow: 0 8px 40px rgba(0,0,0,0.1);
                }}
                .section h2 {{
                    color: #4a5568;
                    margin-bottom: 20px;
                    font-size: 1.8rem;
                    display: flex;
                    align-items: center;
                    gap: 10px;
                }}
                .collectors-grid {{
                    display: grid;
                    grid-template-columns: repeat(auto-fit, minmax(300px, 1fr));
                    gap: 20px;
                }}
                .collector-card {{
                    background: linear-gradient(45deg, #f7fafc, #edf2f7);
                    border-radius: 15px;
                    padding: 20px;
                    cursor: pointer;
                    transition: all 0.3s ease;
                    border: 2px solid transparent;
                }}
                .collector-card:hover {{
                    transform: translateY(-5px);
                    border-color: #667eea;
                }}
                .collector-header {{
                    display: flex;
                    align-items: center;
                    margin-bottom: 15px;
                }}
                .collector-icon {{
                    width: 50px;
                    height: 50px;
                    background: linear-gradient(45deg, #667eea, #764ba2);
                    border-radius: 12px;
                    display: flex;
                    align-items: center;
                    justify-content: center;
                    margin-right: 15px;
                    font-size: 20px;
                    color: white;
                }}
                .captures-table {{
                    width: 100%;
                    border-collapse: collapse;
                    margin-top: 20px;
                }}
                .captures-table th, .captures-table td {{
                    padding: 12px;
                    text-align: left;
                    border-bottom: 1px solid #e2e8f0;
                }}
                .captures-table th {{
                    background: linear-gradient(45deg, #f7fafc, #edf2f7);
                    color: #4a5568;
                    font-weight: 600;
                }}
                .captures-table tr:hover {{
                    background: rgba(102, 126, 234, 0.05);
                    cursor: pointer;
                }}
                .capture-id {{
                    background: linear-gradient(45deg, #667eea, #764ba2);
                    color: white;
                    padding: 4px 8px;
                    border-radius: 8px;
                    font-size: 0.8rem;
                    font-weight: bold;
                }}
                @media (max-width: 768px) {{
                    .header-content {{ flex-direction: column; gap: 15px; }}
                    .stats-overview {{ grid-template-columns: 1fr 1fr; }}
                    .collectors-grid {{ grid-template-columns: 1fr; }}
                }}
            </style>
        </head>
        <body>
            <div class="header">
                <div class="container">
                    <div class="header-content">
                        <a href="/" class="back-btn">⬅ Voltar</a>
                        <div class="company-title">
                            <h1>🏢 {company_name}</h1>
                            <p>ID: {company_id}</p>
                        </div>
                        <div></div>
                    </div>
                </div>
            </div>
            
            <div class="main-content">
                <div class="container">
                    <div class="stats-overview">
                        <div class="stat-card">
                            <div class="icon">📸</div>
                            <span class="number">{total_captures}</span>
                            <div class="label">Total de Capturas</div>
                        </div>
                        <div class="stat-card">
                            <div class="icon">🎯</div>
                            <span class="number">{total_detections}</span>
                            <div class="label">Detecções Realizadas</div>
                        </div>
                        <div class="stat-card">
                            <div class="icon">🤖</div>
                            <span class="number">{total_grd}</span>
                            <div class="label">Análises DINO</div>
                        </div>
                        <div class="stat-card">
                            <div class="icon">🔬</div>
                            <span class="number">{total_florence}</span>
                            <div class="label">Análises Florence</div>
                        </div>
                    </div>
                    
                    <div class="section">
                        <h2>📡 Coletores Ativos</h2>
                        <div class="collectors-grid">
                            {''.join([f'''
                            <div class="collector-card" onclick="window.location.href='/collector/{company_id}/{collector[0]}'">
                                <div class="collector-header">
                                    <div class="collector-icon">📡</div>
                                    <div>
                                        <h3>{collector[0]}</h3>
                                        <p style="color: #718096; font-size: 0.9rem;">Modelo: {collector[1]}</p>
                                    </div>
                                </div>
                                <p style="color: #4a5568; margin-bottom: 10px;">{collector[2] or 'Sem descrição'}</p>
                                <p style="color: #718096; font-size: 0.8rem;">📍 {collector[3]:.6f}, {collector[4]:.6f}</p>
                            </div>
                            ''' for collector in collectors])}
                        </div>
                    </div>
                    
                    <div class="section">
                        <h2>📋 Capturas Recentes</h2>
                        <table class="captures-table">
                            <thead>
                                <tr>
                                    <th>ID</th>
                                    <th>Coletor</th>
                                    <th>Timestamp</th>
                                    <th>Detecções</th>
                                    <th>Ação</th>
                                </tr>
                            </thead>
                            <tbody>
                                {''.join([f'''
                                <tr onclick="window.location.href='/capture/{company_id}/{capture[0]}'">
                                    <td><span class="capture-id">#{capture[0]}</span></td>
                                    <td>{capture[1]}</td>
                                    <td>{capture[2]}</td>
                                    <td>{capture[4]} detecções</td>
                                    <td><button style="background: linear-gradient(45deg, #667eea, #764ba2); color: white; border: none; padding: 5px 10px; border-radius: 5px; cursor: pointer;">Ver Detalhes</button></td>
                                </tr>
                                ''' for capture in recent_captures])}
                            </tbody>
                        </table>
                    </div>
                </div>
            </div>
        </body>
        </html>
        '''
        return html
        
    except Exception as e:
        logger.error(f"Erro no dashboard da empresa: {str(e)}")
        return f"<h1>Erro: {str(e)}</h1>", 500

@app.route('/collector/<company_id>/<collector_id>')
def collector_dashboard(company_id, collector_id):
    """Dashboard detalhado de um coletor"""
    try:
        db_manager = monitoring_system.get_database_manager(company_id)
        
        with sqlite3.connect(db_manager.db_path) as conn:
            cursor = conn.cursor()
            
            # Informações do coletor
            cursor.execute('''
                SELECT model, description, latitude, longitude 
                FROM collectors WHERE id = ? AND company_id = ?
            ''', (collector_id, company_id))
            collector_info = cursor.fetchone()
            
            # Capturas do coletor
            cursor.execute('''
                SELECT id, timestamp, raw_image_path, processed_image_path
                FROM captures 
                WHERE collector_id = ? AND company_id = ?
                ORDER BY created_at DESC
                LIMIT 20
            ''', (collector_id, company_id))
            captures = cursor.fetchall()
            
            # Estatísticas do coletor
            cursor.execute('SELECT COUNT(*) FROM captures WHERE collector_id = ?', (collector_id,))
            total_captures = cursor.fetchone()[0]
            
            cursor.execute('''
                SELECT COUNT(*) FROM original_detections 
                WHERE capture_id IN (SELECT id FROM captures WHERE collector_id = ?)
            ''', (collector_id,))
            total_detections = cursor.fetchone()[0]
        
        html = f'''
        <!DOCTYPE html>
        <html lang="pt-BR">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Coletor {collector_id} - Dashboard</title>
            <style>
                * {{ margin: 0; padding: 0; box-sizing: border-box; }}
                body {{ 
                    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    min-height: 100vh;
                    color: #333;
                }}
                .header {{
                    background: rgba(255,255,255,0.95);
                    backdrop-filter: blur(10px);
                    padding: 20px 0;
                    box-shadow: 0 4px 20px rgba(0,0,0,0.1);
                }}
                .container {{ max-width: 1200px; margin: 0 auto; padding: 0 20px; }}
                .header-content {{
                    display: flex;
                    justify-content: space-between;
                    align-items: center;
                }}
                .back-btn {{
                    background: linear-gradient(45deg, #667eea, #764ba2);
                    color: white;
                    padding: 10px 20px;
                    border: none;
                    border-radius: 25px;
                    cursor: pointer;
                    text-decoration: none;
                    transition: all 0.3s ease;
                }}
                .collector-title h1 {{
                    color: #4a5568;
                    font-size: 2rem;
                    margin-bottom: 5px;
                }}
                .main-content {{ padding: 30px 20px; }}
                .info-section {{
                    background: rgba(255,255,255,0.95);
                    backdrop-filter: blur(10px);
                    border-radius: 20px;
                    padding: 30px;
                    margin-bottom: 30px;
                    box-shadow: 0 8px 40px rgba(0,0,0,0.1);
                }}
                .captures-grid {{
                    display: grid;
                    grid-template-columns: repeat(auto-fill, minmax(300px, 1fr));
                    gap: 20px;
                    margin-top: 20px;
                }}
                .capture-card {{
                    background: rgba(255,255,255,0.95);
                    border-radius: 15px;
                    padding: 20px;
                    box-shadow: 0 8px 30px rgba(0,0,0,0.1);
                    cursor: pointer;
                    transition: all 0.3s ease;
                }}
                .capture-card:hover {{
                    transform: translateY(-5px);
                    box-shadow: 0 15px 40px rgba(0,0,0,0.15);
                }}
                .capture-header {{
                    display: flex;
                    justify-content: space-between;
                    align-items: center;
                    margin-bottom: 15px;
                }}
                .capture-id {{
                    background: linear-gradient(45deg, #667eea, #764ba2);
                    color: white;
                    padding: 5px 10px;
                    border-radius: 10px;
                    font-weight: bold;
                }}
                .capture-time {{
                    color: #718096;
                    font-size: 0.9rem;
                }}
            </style>
        </head>
        <body>
            <div class="header">
                <div class="container">
                    <div class="header-content">
                        <a href="/company/{company_id}" class="back-btn">⬅ Voltar</a>
                        <div class="collector-title">
                            <h1>📡 Coletor {collector_id}</h1>
                            <p>Modelo: {collector_info[0] if collector_info else 'N/A'}</p>
                        </div>
                        <div></div>
                    </div>
                </div>
            </div>
            
            <div class="main-content">
                <div class="container">
                    <div class="info-section">
                        <h2>ℹ️ Informações do Coletor</h2>
                        <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 20px; margin-top: 20px;">
                            <div>
                                <strong>Modelo:</strong> {collector_info[0] if collector_info else 'N/A'}<br>
                                <strong>Descrição:</strong> {collector_info[1] if collector_info and collector_info[1] else 'Não informado'}<br>
                                <strong>Localização:</strong> {f"{collector_info[2]:.6f}, {collector_info[3]:.6f}" if collector_info else 'N/A'}
                            </div>
                            <div>
                                <strong>Total de Capturas:</strong> {total_captures}<br>
                                <strong>Total de Detecções:</strong> {total_detections}
                            </div>
                        </div>
                    </div>
                    
                    <div class="info-section">
                        <h2>📸 Capturas Realizadas</h2>
                        <div class="captures-grid">
                            {''.join([f'''
                            <div class="capture-card" onclick="window.location.href='/capture/{company_id}/{capture[0]}'">
                                <div class="capture-header">
                                    <span class="capture-id">#{capture[0]}</span>
                                    <span class="capture-time">{capture[1]}</span>
                                </div>
                                <div>
                                    <p><strong>📁 Arquivo RAW:</strong> {os.path.basename(capture[2])}</p>
                                    <p><strong>🎨 Arquivo Processado:</strong> {os.path.basename(capture[3])}</p>
                                </div>
                            </div>
                            ''' for capture in captures])}
                        </div>
                    </div>
                </div>
            </div>
        </body>
        </html>
        '''
        return html
        
    except Exception as e:
        logger.error(f"Erro no dashboard do coletor: {str(e)}")
        return f"<h1>Erro: {str(e)}</h1>", 500

In [ ]:
if __name__ == '__main__':
    os.makedirs(BASE_DATA_DIR, exist_ok=True)
    app.run(host='127.0.0.1', port=2020, debug=True, use_reloader=False)